## Ejemplo, 3 entradas y 2 salidas
Se necesita está creando un sistema para clasificar los tickets de emisión de los clientes por prioridad y enrutarlos al departamento correcto, el modelo tendrá tres entradas:

el título del ticket (entrada de texto),
el cuerpo del texto del ticket (entrada de texto), y
cualquier etiqueta agregada por el usuario (entrada categórica)
Este modelo tendrá dos salidas:

la puntuación de prioridad entre 0 y 1 (salida sigmoidea escalar), y
el departamento que debe manejar el ticket (salida softmax sobre el conjunto de departamentos).

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

num_tags = 12 
num_departments = 4 
num_words = 10000  # Size of vocabulary obtained when preprocessing text data

In [4]:
title_input = keras.Input(shape=(None,), name="title")  # Variable-length sequence of ints

body_input = keras.Input(shape=(None,), name="body")  

tags_input = keras.Input(shape=(num_tags,), name="tags") # Binary vectors of size `num_tags`

In [5]:
print(title_input)

KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name='title'), name='title', description="created by layer 'title'")


In [9]:
# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input) 
body_features = layers.Embedding(num_words, 64)(body_input)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
title_features = layers.LSTM(128)(title_features) 
body_features = layers.LSTM(32)(body_features)
# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input])

In [13]:
# Stick a logistic regression for priority prediction on top of the features
priority_pred = layers.Dense(1, name="priority")(x)
# Stick a department classifier on top of the features
department_pred = layers.Dense(num_departments, name="department")(x)

In [14]:
print(priority_pred)
print(department_pred)

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='priority/BiasAdd:0', description="created by layer 'priority'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='department/BiasAdd:0', description="created by layer 'department'")


In [15]:
model = keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred],
)

In [16]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[
        keras.losses.BinaryCrossentropy(from_logits=True),
        keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    loss_weights=[1.0, 0.2],
)

In [17]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": keras.losses.BinaryCrossentropy(from_logits=True),
        "department": keras.losses.CategoricalCrossentropy(from_logits=True),
    },
    loss_weights={"priority": 1.0, "department": 0.2},
)

In [18]:
# Simular valores de entradas
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype("float32")

In [21]:
# Simular respuestas validas
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

In [22]:
model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_targets, "department": dept_targets},
    epochs=2,
    batch_size=32,
)

Epoch 1/2
40/40 [==============================] - 3s 19ms/step - loss: 1.2838 - priority_loss: 0.6997 - department_loss: 2.9209
Epoch 2/2
40/40 [==============================] - 1s 18ms/step - loss: 1.3030 - priority_loss: 0.6959 - department_loss: 3.0352
